In [30]:
def convert_ls_json(json_path, mappings, label_dir,  img_dir = None):
    with open(json_path, "r") as f:
        data = json.load(f)

    img_path = data['task']['data']['image'].replace("/data/local-files/?d=", "/home/tasanders/")
    img_name = os.path.basename(img_path) 
    txt_name = os.path.basename(img_path).replace(".jpg", ".txt")

    new_annots = []
    annotations = data['result']

    for annot in annotations:
        img_width, img_height = annot['original_width'], annot['original_height']
        label = annot['value']['rectanglelabels'][0]
        label_id = mappings[label]
        x = (annot['value']['x'] + annot['value']['width']/2) / 100
        y = (annot['value']['y'] + annot['value']['height']/2) / 100
        w = annot['value']['width'] / 100
        h = annot['value']['height'] / 100

        new_annots.append(f"{label_id} {x} {y} {w} {h}")

    with open(os.path.join(label_dir, txt_name), "w") as f:
        f.write("\n".join(new_annots))

    if img_dir:
        shutil.copy(img_path, os.path.join(img_dir, img_name))

In [31]:
convert_ls_json(test_file, mappings, "/home/tasanders/GitHub/SquareEyes/datasets/Custom/validation_study/training/labels", "/home/tasanders/GitHub/SquareEyes/datasets/Custom/validation_study/training/images")

In [75]:
import sys
sys.path.append('../')

from src.utils.utils import make_classes_file

from pathlib import Path
import os
import shutil
from tqdm.notebook import tqdm
from label_studio_converter.imports import yolo
import json


In [58]:
dir = Path("../datasets/coco")
convert_dir = dir


In [107]:
temp_dir = dir/"temp"
temp_dir.mkdir(parents=True, exist_ok=True)
for p in "images", "labels":
    (temp_dir / p).mkdir(parents=True, exist_ok=True)

images = [file for file in (dir / "images").rglob("*.jpg")]

for image in tqdm(images, desc="Copying images", unit="file"):
    dest_file_path = temp_dir / "images" / image.name
    shutil.copy(image, dest_file_path)

labels = [file for file in (dir / "labels").rglob("*.txt")]

for label in tqdm(labels, desc="Copying labels", unit="file"):
    dest_file_path = temp_dir / "labels" / label.name
    shutil.copy(label, dest_file_path)

Copying images:   0%|          | 0/14610 [00:00<?, ?file/s]

Copying labels:   0%|          | 0/13541 [00:00<?, ?file/s]

In [108]:
convert_dir = str(temp_dir)
name = "coco"

if not os.path.exists(convert_dir + "/classes.txt"):
    make_classes_file(convert_dir)

In [109]:
yolo.convert_yolo_to_ls(
    input_dir=convert_dir,
    out_file=convert_dir + f"/{name}-tasks.json",
    image_root_url="/data/local-files/?d=GitHub/SquareEyes/" + str(dir).replace("../","") + "/images",
    out_type="predictions",
)


  1. Create a new project in Label Studio
  2. Use Labeling Config from "../datasets/coco/temp/coco-tasks.label_config.xml"
  3. Setup serving for images [e.g. you can use Local Storage (or others):
     https://labelstud.io/guide/storage.html#Local-storage]
  4. Import "../datasets/coco/temp/coco-tasks.json" to the project



In [111]:
# Read in the json file
with open(convert_dir + f"/{name}-tasks.json", "r") as f:
    data = json.load(f)

In [112]:
for task in tqdm(data):
    curr_img = task['data']['image']
    img_basename = os.path.basename(curr_img)

    path_index = next((i for i, p in enumerate(images) if p.name == img_basename), None)

    if path_index is not None:
        subsection = images.pop(path_index).parts[-2]

        new_img = curr_img.replace(f"/images/{img_basename}", f"/images/{subsection}/{img_basename}")

        task['data']['image'] = new_img  

with open(convert_dir + f"/{name}-tasks.json", "w") as f:
    json.dump(data,f)



  0%|          | 0/14610 [00:00<?, ?it/s]

In [103]:
curr_img = data[1]['data']['image']
img_basename = os.path.basename(curr_img)

In [104]:
path_index = next((i for i, p in enumerate(images) if p.name == img_basename), None)
if path_index is not None:
    subsection = images.pop(path_index).parts[-2]

    new_img = curr_img.replace(f"/images/{img_basename}", f"/images/{subsection}/{img_basename}")
    


In [110]:
images

[PosixPath('../datasets/coco/images/train2017/000000159889.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000345855.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000241355.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000008999.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000574678.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000066164.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000121472.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000295558.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000451142.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000523487.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000122235.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000562835.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000215398.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000516866.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000553506.jpg'),
 PosixPath

In [91]:
data[1]

{'data': {'image': '/data/local-files/?d=GitHub/SquareEyes/datasets/coco/images/000000345855.jpg'},
 'predictions': [{'result': [{'id': '965eba4194',
     'type': 'rectanglelabels',
     'value': {'x': 45.57975,
      'y': 29.491849999999996,
      'width': 12.6609,
      'height': 29.7377,
      'rotation': 0,
      'rectanglelabels': ['Laptop Computer']},
     'to_name': 'image',
     'from_name': 'label',
     'image_rotation': 0,
     'original_width': 640,
     'original_height': 427}],
   'ground_truth': False}]}

In [85]:
import glob

In [83]:
glob.glob()

[PosixPath('../datasets/coco/images/train2017/000000159889.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000345855.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000241355.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000008999.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000574678.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000066164.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000121472.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000295558.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000451142.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000523487.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000122235.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000562835.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000215398.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000516866.jpg'),
 PosixPath('../datasets/coco/images/train2017/000000553506.jpg'),
 PosixPath